In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import re
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import os
from pdb import set_trace

In [2]:
def load_data(n_months=15):
    start_date = datetime.datetime.today()
    fraud_df_list = []
    GraphSIF_df_list = []

    for i in range(n_months):
        start_date += relativedelta(months = -1)
        mon = f'{str(start_date.month).zfill(2)}{start_date.year}'
        print(mon)
        if os.path.exists(f'./static/Fraud1_Flags_{mon}.csv'): fraud_df_list.append(pd.read_csv(f'./static/Fraud1_Flags_{mon}.csv'))
        if os.path.exists(f'./static/GraphSIFresults_{mon}.csv'): GraphSIF_df_list.append(pd.read_csv(f'./static/GraphSIFresults_{mon}.csv'))
    fraud_df = pd.concat(fraud_df_list)
    graphSIF_df = pd.concat(GraphSIF_df_list)
    return fraud_df,graphSIF_df

In [3]:
fraud_df,graphSIF_df = load_data()

112021
102021
092021
082021
072021
062021
052021


/home/skumar/miniconda3/envs/AP_Flask/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (25,70,104,114,115,121,211) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/skumar/miniconda3/envs/AP_Flask/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (70,104,114,115,121,211) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


042021
032021
022021
012021
122020
112020
102020
092020


In [10]:
q = fraud_df.Quarter.unique()
quats = fraud_df.Quarter.unique()
quats.sort()


quarters
fraud_df[fraud_df.Quarter.isin(quats[-2:])].Quarter.unique()

array(['2021_Q2', '2021_Q1'], dtype=object)

In [21]:
tmp = fraud_df[['Quarter','VENDOR_ID','VENDOR_NAME_SHORT','ACCOUNT_CAT','BUSINESS_UNIT','Full_Address','Vendor_PGRK','Account_PGRK','BU_PGRK','Address_PGRK','Vendor_PR_Var', 'Account_PR_Var', 'BU_PR_Var', 'Address_PR_Var']]

In [22]:
tmp.head()

,Quarter,VENDOR_ID,VENDOR_NAME_SHORT,ACCOUNT_CAT,BUSINESS_UNIT,Full_Address,Vendor_PGRK,Account_PGRK,BU_PGRK,Address_PGRK,Vendor_PR_Var,Account_PR_Var,BU_PR_Var,Address_PR_Var
0,2021_Q2,ACEUS0000068129,LIGHTEDGES-001,PRE_T_INC_INCL_RL_GN,10508,"PO BOX 874074, KANSAS CITY, MO, USA",0.890443,592.637303,20.076933,0.419444,0.064349,4.624858,-4.645429,0.019581
1,2021_Q2,ACEUS0000096529,JWHAMMERLL-001,PRE_T_INC_INCL_RL_GN,10008,"4609 LYNHURST RD, SPRINGFIELD, IL, USA",0.886814,592.637303,588.321512,0.418136,0.662981,4.624858,3.126719,0.432447
2,2021_Q2,ACECA0000005661,SERVICESDE-001,PRE_T_INC_INCL_RL_GN,72061,"AUTOMOBILES CANADIENS INC, PO BOX 20039 RPO BA...",0.749724,592.637303,27.115617,0.377596,-21.068945,4.624858,3.384136,-13.662313
3,2021_Q2,ACEUS0000002800,ESISINC-001,PRE_T_INC_INCL_RL_GN,10008,"DEPT CH 10123, PALATINE, IL, USA",28.804388,592.637303,588.321512,5.612262,16.031197,4.624858,3.126719,7.539529
4,2021_Q2,ACECA0000007384,MANIONWILK-001,OTHER_ASSETS,32002,"500 21 FOUR SEASONS PLACE, TORONTO, ON, CAN",0.597571,82.807017,8.024211,0.334669,NaN,-2.116058,-13.092493,NaN


## Input data Prep (PageRank and Edges)

In [25]:
def GraphInp_prep(fraud_df):
    
    ### PageRank df preparation
    pr_stats_df = fraud_df[['Quarter','VENDOR_ID','NAME1','ACCOUNT_CAT','BUSINESS_UNIT','Full_Address','Vendor_PGRK','Account_PGRK','BU_PGRK','Address_PGRK','Vendor_PR_Var', 'Account_PR_Var', 'BU_PR_Var', 'Address_PR_Var']].copy()

    dic = {'VENDOR_ID': ['Quarter','VENDOR_ID','Vendor_PGRK','Vendor_PR_Var','NAME1'],
           'ACCOUNT': ['Quarter','ACCOUNT_CAT','Account_PGRK','Account_PR_Var','NAME1'],
           'BUSINESS_UNIT': ['Quarter','BUSINESS_UNIT','BU_PGRK','BU_PR_Var','NAME1'],
           'Full_Address': ['Quarter','Full_Address','Address_PGRK','Address_PR_Var','NAME1']}

    PageRank_df = pd.DataFrame()
    for x, y in dic.items():
        entity_df = pr_stats_df[y]
        if not x == 'VENDOR_ID':
            entity_df['NAME1'] = ''
        entity_df['Entity_type'] = x
        entity_df.columns = ['Quarter', 'id', 'PageRank','PGRK_PrctCng','NAME1', 'Entity_type']
        entity_df_agg = entity_df.groupby(
            list(entity_df.columns),dropna=False).size().reset_index(name='n_trans')
        PageRank_df = PageRank_df.append(entity_df_agg)

    PageRank_df = pd.pivot_table(PageRank_df,
                                  index=['id', 'Entity_type', 'NAME1'],
                                  columns=['Quarter'],
                                  values=['PageRank','PGRK_PrctCng', 'n_trans']).reset_index()
    PageRank_df.columns = ['_'.join(col).strip('_') for col in PageRank_df.columns]
    PageRank_df['id'] = PageRank_df.id.apply(lambda x: re.sub(', ', ',<br>', x, 2))

    ## Edges df preparation
    Edges_stats = fraud_df[['VENDOR_ID', 'Full_Address', 'ACCOUNT_CAT', 'BUSINESS_UNIT','Quarter']].copy()

    Edges_stats = Edges_stats.melt(id_vars=['Quarter','VENDOR_ID'], value_vars=['ACCOUNT_CAT', 'BUSINESS_UNIT','Full_Address'])

    Edges_df = Edges_stats.groupby(list(Edges_stats.columns),dropna=False).size().reset_index(name='n_edges')
    Edges_df.drop('variable',axis=1,inplace=True)
    Edges_df.rename(columns={'VENDOR_ID':'src','value':'dst'},inplace=True)
    Edges_df['dst'] = Edges_df.dst.apply(lambda x: re.sub(', ', ',<br>', x, 2))

    return PageRank_df, Edges_df


In [28]:
pg, ed = GraphInp_prep(fraud_df)

/home/skumar/miniconda3/envs/AP_Flask/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/skumar/miniconda3/envs/AP_Flask/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### PageRank stats

In [29]:
pg.sample(10)

,id,Entity_type,NAME1,PGRK_PrctCng_2021_Q1,PGRK_PrctCng_2021_Q2,PageRank_2021_Q1,PageRank_2021_Q2,n_trans_2021_Q1,n_trans_2021_Q2
7340,ACEUS0000097859,VENDOR_ID,OMAR TANBOUZ,NaN,NaN,NaN,0.586991,NaN,1.0
9897,USCBL0000317582,VENDOR_ID,"BROWN & BROWN OF NEW YORK, INC.",NaN,2.793549,0.665201,0.683784,1.0,1.0
3728,ACECA0000005248,VENDOR_ID,EXAMONE CANADA,-40.334700,-26.499230,1.306721,0.960450,4.0,3.0
6264,ACEUS0000089858,VENDOR_ID,THE COTTON CHARITABLE FOUNDATION,NaN,NaN,NaN,0.556990,NaN,1.0
1855,"3330 CUMBERLAND BLVD SUITE 500,<br>ATLANTA,<br...",Full_Address,,NaN,NaN,NaN,0.328778,NaN,1.0
5199,ACEUS0000062309,VENDOR_ID,CAPITOL CITY MINI STORAGE,-4.058044,0.064349,0.889871,0.890443,3.0,3.0
1885,"3427 POLO DOWNS,<br>HOOVER,<br>AL, USA",Full_Address,,NaN,NaN,NaN,0.431689,NaN,2.0
2835,"7 ZUILLS PARK DRIVE,<br>SMITHS,<br>BMU",Full_Address,,-4.527394,1.498969,0.435419,0.441946,3.0,3.0
9825,"TOMORROWS VOICES,<br>#4 HAPPY VALLEY ROAD,<br>...",Full_Address,,NaN,NaN,NaN,0.330256,NaN,1.0
5297,ACEUS0000065807,VENDOR_ID,"SF&C INSURANCE ASSOCIATES, INC",-39.201160,14.682237,1.126653,1.292070,4.0,5.0


### Entity Edges

In [31]:
ed.sample(15)

,Quarter,src,dst,n_edges
10418,2021_Q1,USCBL0002870035,"1440 CHAPIN AVE SUITE 205,<br>BURLINGAME,<br>C...",1
161,2021_Q1,ACEBM0000002025,81200,1
21436,2021_Q2,USCBL0002870371,USBFA,1
14928,2021_Q2,ACEUS0000058213,70012,2
7037,2021_Q1,ACEUS0000089873,"852 HART ST,<br>BROOKLYN,<br>NY, USA",2
10100,2021_Q1,USCBL0002869093,"1111 BRIARCREST DRIVE,<br>BRYAN,<br>TX, USA",1
6285,2021_Q1,ACEUS0000083631,PRE_T_INC_INCL_RL_GN,2
10892,2021_Q2,ACEBM0000001411,PRE_T_INC_INCL_RL_GN,1
6486,2021_Q1,ACEUS0000085689,"403B GERMAN MOYER URB,<br>PARQUE CENTRAL,<br>S...",6
7460,2021_Q1,ACEUS0000093558,"5 WESTBROOK CORPORATE CENTER,<br>STE 920,<br>W...",3


In [30]:
list(Quarters_options.keys())[0]

NameError: name 'Quarters_options' is not defined

## Appendix

In [37]:
list(range(1,n_top+1))

[1, 2, 3]

In [18]:
graphSIF_df.head()

,GraphSIF_BUs,Month,Is_anomolous,top_feature_1,top_feature_2,top_feature_3,top_feature_4,top_feature_5,Hist_average_1,Hist_average_2,Hist_average_3,Hist_average_4,Hist_average_5,Test_hstgm_1,Test_hstgm_2,Test_hstgm_3,Test_hstgm_4,Test_hstgm_5
0,Bu_Monthly_0_10,2021_6,0.0,1_2_1_Total_Amt,1_1_1_Total_Amt,1_1_1_Pattern_Count,1_1_3_Total_Amt,2_1_1_Total_Amt,4.858343e+03,2.747677e+07,38.375000,286017.250000,188046.050995,1.054517e+05,7.181902e+07,25.00000,0.0,1500.0
1,Bu_Monthly_20_30,2021_6,0.0,1_2_1_Pattern_Count,1_1_2_Pattern_Count,1_1_1_Pattern_Count,1_4_1_Pattern_Count,1_1_2_Total_Amt,3.333333e+00,1.458333e+00,149.000000,0.500000,16647.963526,0.000000e+00,0.000000e+00,123.00000,0.0,0.0
2,10028,2021_6,0.0,1_1_1_Total_Amt,1_1_1_Pattern_Count,2_1_1_Total_Amt,2_1_1_Pattern_Count,1_6_1_Total_Amt,4.324272e+06,4.429167e+01,671833.200414,3.625000,0.000000,2.830424e+06,5.000000e+00,0.00000,0.0,0.0
3,10100,2021_6,0.0,1_1_1_Pattern_Count,1_1_1_Total_Amt,1_2_1_Total_Amt,1_2_1_Pattern_Count,1_7_1_Total_Amt,1.570833e+01,9.668485e+04,175.563751,0.041667,0.000000,2.200000e+01,1.140626e+05,0.00000,0.0,0.0
4,USBFA,2021_6,0.0,1_2_1_Total_Amt,1_2_1_Pattern_Count,1_1_1_Total_Amt,1_1_1_Pattern_Count,1_1_2_Pattern_Count,8.268792e+02,1.666667e-01,140854.378731,31.041667,2.416667,8.604990e+03,1.000000e+00,73148.89035,29.0,2.0


In [62]:
month = '2021_4'
BU = '10028'
Is_anomolous = 0

feat_labels = {'Pattern_Count':"transaction volume",
               'Total_Amt':"total amount"}

In [74]:
def get_GraphSIF_stats(BU,month):
    graphSIF_sub = graphSIF_df[(graphSIF_df.Month == month) &
                               (graphSIF_df.GraphSIF_BUs == BU)].squeeze()
    
    ## get top feature stats
    top_feat_values = {graphSIF_sub[f'top_feature_{i}']:[graphSIF_sub[f'Hist_average_{i}'],graphSIF_sub[f'Test_hstgm_{i}']]  for i in range(1,n_top+1)}

    stats_text = f'Changes observed in the top 3 features for test month (histogram) compared to historical 24 month average for BU {BU} in {month} month'
    for key, value in top_feat_values.items():
        feat, avg, test_hist = key, round(value[0],2), round(value[1],2)
        feat_split = feat.split('_')
        n_acc,n_add,n_ven, feat_key = feat_split[0],feat_split[1],feat_split[2],'_'.join(feat_split[3:])
        prct_chng = (test_hist-avg)/avg*100
        stats_text += f"\n {feat}: {str(abs(round(prct_chng,2)))}% {'decrease' if prct_chng < 0 else 'increase'} with respect to past 24 months average ({avg}) in the {feat_labels[feat_key]} with the pattern where {n_ven} vendor connected has {n_add} addresses and is transacting under {n_acc} accounts. current month {feat_labels[feat_key]} is {test_hist}"
    return stats_text

In [76]:
fraud_df

(34555, 216)

In [23]:
month = '2021_4'
BU = '10028'

In [ ]:
fraud_df_sub = fraud_df[(fraud_df.Month = month) & (flags_df.GraphSIF_BUs == BU)]
Flag_cols = ['BU_Is_Multi_VendorAddress',
             'BU_Is_Multi_AccCat',
             'BU_Is_Address_MultiVendor']

In [8]:
# BUs to be displayed in the dropdown menu
BUs = graphSIF_df['GraphSIF_BUs'].unique()
BUs.sort()
BU_options = [{'label': i, 'value': i} for i in BUs]
BUs[0]

'10008'

In [ ]:
layout = html.Div([
    dbc.Container([
        dbc.Row([html.H3(children='GraphSIF Analysis')]),
        dbc.Row([
            dbc.Col([
                html.Div([html.Label('Select Month'),
                          dcc.Dropdown(id='select-month',
                                       options=Month_options,
                                       value=Mnths[0],
                                       clearable=False),
                          dcc.Graph(id='monthly-bu-flags')])
            ], id='top-vendors', width=3),
            dbc.Col([
                html.Div([
                    dbc.Row([
                        html.Div([
                            html.Label('Select BU'),
                            dcc.Dropdown(
                                id='select-BU',
                                options=BU_options,
                                value=BUs[0],
                                clearable=False)],
                            style={'paddingLeft': '15px',
                                   'paddingRight': '15px'}),
                        html.Div([
                            html.Label('Is Anomalous'),
                            dcc.Dropdown(
                                id='is-anomalous',
                                options=Anom_options,
                                value=0,
                                clearable=False)])
                    ]),
                    dbc.Row([html.H6(id='graph-sif-writeup',
                                     style={"margin-top": "5px"})])
                ], style={"margin-left": "45px"})
            ])
        ])
    ])
])

In [ ]:
@app.callback(
    Output('graph-sif-writeup', 'children'),
    Input('select-BU', 'value'),
    State('select-month', 'value'))
def display_GraphSIF_stats(BU,month):
    return get_GraphSIF_stats(BU,month)

In [ ]:
@app.callback(
    Output('select-BU', 'options'),
    Output('select-BU', 'value'),
    Input('is-anomalous', 'value'),
    State('select-month', 'value'))
def update_bu_option(is_anomulus, month):
    BU_list = (graphSIF_df[(graphSIF_df.Is_anomolous == is_anomulus) & 
                           (graphSIF_df.Month == month)]
               .GraphSIF_BUs.values)
    BU_list.sort()
    return [{'label': BU, 'value': BU} for BU in BU_list],BU_list[0]

In [10]:
graphSIF_df.sample(5)

,GraphSIF_BUs,Month,Is_anomolous,top_feature_1,top_feature_2,top_feature_3,top_feature_4,top_feature_5,Hist_average_1,Hist_average_2,Hist_average_3,Hist_average_4,Hist_average_5,Test_hstgm_1,Test_hstgm_2,Test_hstgm_3,Test_hstgm_4,Test_hstgm_5
8,10008,2021_1,0.0,2_2_1_Total_Amt,4_6_1_Pattern_Count,1_1_2_Pattern_Count,2_2_1_Pattern_Count,2_5_2_Pattern_Count,8.181376e+06,0.083333,1.816667e+01,11.791667,2.500000e-01,3.895896e+07,1.0,9.000000e+00,19.000000,1.000000
13,10118,2021_3,0.0,1_1_1_Total_Amt,1_1_3_Total_Amt,1_2_1_Pattern_Count,1_2_1_Total_Amt,1_1_1_Pattern_Count,7.457953e+06,479882.375458,8.333333e-01,551410.289131,1.187500e+01,1.924261e+07,266250.0,1.000000e+00,590269.675903,14.000000
0,USBFJ,2021_5,0.0,1_1_2_Total_Amt,1_1_2_Pattern_Count,1_1_1_Pattern_Count,1_2_1_Total_Amt,1_1_1_Total_Amt,0.000000e+00,0.000000,2.829167e+01,1278.370827,1.083497e+05,8.538390e+03,4.0,6.100000e+01,8277.510010,181570.399323
8,CAGCA,2021_6,0.0,1_1_2_Pattern_Count,1_2_1_Pattern_Count,2_2_1_Pattern_Count,2_1_2_Pattern_Count,2_1_1_Pattern_Count,2.916667e+00,1.375000,3.750000e-01,0.500000,1.208333e+00,9.000000e+00,0.0,1.000000e+00,1.000000,0.000000
8,70012,2021_4,0.0,2_1_1_Pattern_Count,2_3_1_Pattern_Count,1_1_1_Total_Amt,1_1_2_Pattern_Count,2_1_1_Total_Amt,5.083333e+00,0.083333,2.499954e+07,3.041667,1.375964e+06,1.600000e+01,1.0,1.673729e+07,6.000000,335905.439087


In [ ]:
@app.callback(
    Output('is-anomalous', 'value'),
    Input('select-month', 'value'))
def update_anom_value(month):
    return 0

In [ ]:
@ app.callback(
    Output('monthly-bu-flags', 'figure'),
    Input('select-BU', 'value'),
    State('start-month', 'value'))
def create_monthly_flags_chart(BU, month):
    return monthly_flags_chart(BU, month)

In [ ]:

def Flag_prop_plot(BU, start, end):
    if BU != 'ALL':
        flags_df_sub = flags_df[(flags_df.BUSINESS_UNIT == BU) &
                                (flags_df.Month >= start) &
                                (flags_df.Month <= end)]
    else:
        flags_df_sub = flags_df[(flags_df.Month >= start) &
                                (flags_df.Month <= end)]

    flags_prop = flags_df_sub[Flag_cols].sum()

    flags_prop_df = flags_prop.to_frame().reset_index()
    flags_prop_df.columns = ['flag_label', 'count']
    flags_prop_df['count'] = flags_prop_df['count']
    flags_prop_df = flags_prop_df.round(2)
    flags_prop_df['label_name'] = flags_prop_df.flag_label.replace(
        flag_lab2name)

    fig = px.bar(flags_prop_df, y='count', x='label_name',
                 labels={'label_name': 'Scenario',
                         'count': 'Transaction count'},
                 color='label_name',
                 color_discrete_map=color_dict)

    fig.update_layout(transition_duration=500,
                      title=None,
                      autosize=True,
                      xaxis=dict(visible=False, showgrid=False,
                                 zeroline=False, showticklabels=False),
                      margin=dict(t=20, l=30, r=0, b=0),
                      width=1250)
    return fig

In [ ]:
[col for col in fraud_df.columns .]

In [78]:
Flag_cols = ['BU_Is_Multi_VendorAddress',
             'BU_Is_Multi_AccCat',
             'BU_Is_Address_MultiVendor']

BU = '10008'

flag_lab2name = {'BU_Is_Multi_VendorAddress': 'Transaction Volume where vendor is connected to Multiple Addresses under same BU in a month',
                 'BU_Is_Multi_AccCat': 'Transaction Volume where Vendor is connected to Multiple Accounts under same BU in a month',
                 'BU_Is_Address_MultiVendor': 'Transaction Volume where Address is connected to Multiple vendors under same BU in a month'}

colorlist = ['#636EFA',
             '#EF553B',
             '#00CC96']
             
color_dict = {x: y for x, y in zip(flag_lab2name.values(), colorlist)}

import plotly.express as px


In [ ]:
def get_GraphSIF_stats(BU, month):
    graphSIF_sub = graphSIF_df[(graphSIF_df.Month == month) &
                               (graphSIF_df.GraphSIF_BUs == BU)].squeeze()

    # get top feature stats
    top_feat_values = {graphSIF_sub[f'top_feature_{i}']: [
        graphSIF_sub[f'Hist_average_{i}'], graphSIF_sub[f'Test_hstgm_{i}']] for i in range(1, n_top+1)}

    stats_list = [f'Changes observed in the top 3 features for test month (histogram) compared to historical 24 month average for BU {BU} in {month} month']
    for key, value in top_feat_values.items():
        feat, avg, test_hist = key, round(value[0], 2), round(value[1], 2)
        feat_split = feat.split('_')
        n_acc, n_add, n_ven, feat_key = feat_split[0], feat_split[1], feat_split[2], '_'.join(
            feat_split[3:])
        prct_chng = (test_hist-avg)/avg*100
        stats_list.append(html.Br())
        stats_list.append(f"{feat}: {str(abs(round(prct_chng,2)))}% {'decrease' if prct_chng < 0 else 'increase'} with respect to past 24 months average ({avg}) in the {feat_labels[feat_key]} with the pattern where {n_ven} vendor connected has {n_add} addresses and is transacting under {n_acc} accounts. current month {feat_labels[feat_key]} is {test_hist}")
    return stats_list

In [34]:
(html.Div(html.P([f'Transaction volume increase for {Vendor} from {prev_quarter} to {quarter}: {prcnt}%',
                             html.Br(),
                             html.Br(),
                             f'Transactions for vendor {Vendor} jumped from {prv_vol} in {prev_quarter} to {curr_vol} in {quarter}'])))

In [86]:
def monthly_flags_chart(BU, month):    
    flags_df_sub = flags_df.loc[(fraud_df.GraphSIF_BUs==BU)&(fraud_df.Month==month),Flag_cols]

    flags_count = flags_df_sub[Flag_cols].sum()

    flags_count_df = flags_count.to_frame().reset_index()
    flags_count_df.columns = ['flag_label', 'count']

    flags_count_df['label_name'] = flags_count_df.flag_label.replace(flag_lab2name)


    fig = px.bar(flags_count_df, y='count', x='label_name',
                 labels={'label_name': 'BU flags',
                         'count': 'Transaction count'},
                 color='label_name',
                 color_discrete_map=color_dict)

    fig.update_layout(transition_duration=500,
                      legend = dict(yanchor='bottom',
                                    y=1.0,
                                    x=-0.1),
                      title=f"Monthly flags for BU {BU} in {month} month",
                      autosize=True,
                      xaxis=dict(visible=False, showgrid=False,
                                 zeroline=False, showticklabels=False),
                      margin=dict(t=175, l=0, r=0, b=0),
                      width=700)
    return fig
monthly_flags_chart(BU, month)